# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Set the style of the plot
sns.set_style("whitegrid")

# Import dataset

In [ ]:
# Import dataframe
df = pd.read_excel('novak_djokovic_historial.xlsx')

# Pre-processing

In [ ]:
# Transform Date to datetime
df['Date'] = pd.to_datetime(df['Date'].str.replace('‑', '-'), format='%d-%b-%Y')

In [ ]:
# Clean result column before splitting
df['Result'] = df['Result'].str.replace(r'\([^)]*\)', '').str.replace(r'\[[^\]]*\]', '')

In [ ]:
# Split winner from loser in column result
split_columns = df['Result'].str.split(r'd\.', expand=True)
df['Winner'] = split_columns[0].str.replace('\xa0', '')
df['Loser'] = split_columns[1].str.replace('\xa0', '')


In [ ]:
# Create winner flag
df['winner_flag'] = np.where(df['Winner'].str.contains('Djokovic'), 'Win', 'Lose')

In [ ]:
# Count number of sets in the match
df['sets_number'] = df['Score'].str.count('-')

In [ ]:
# All history
df.winner_flag.value_counts()

In [ ]:
df.Tournament.sort_values().unique()

In [ ]:
# Create column of grand_slam_flag
grand_slams = ['Roland Garros','Australian Open', 'US Open','Wimbledon']
df['grand_slam_flag'] = np.where(df['Tournament'].isin(grand_slams), True, False)

In [ ]:
# Historial grand slams
df[df['grand_slam_flag']]['winner_flag'].value_counts()

# Grand slam Analysis ##

## Historial general de partidos

In [ ]:
df_grand_slam = df[df['grand_slam_flag']==True]

In [ ]:
df_grand_slam_plot = df_grand_slam.groupby(['Tournament','winner_flag']).agg(result=('Winner','count')).reset_index()

In [ ]:
# Calculate the total for each tournament
df_grand_slam_plot['Total'] = df_grand_slam_plot.groupby('Tournament')['result'].transform('sum')

# Calculate the percentage of Win and Lose for each tournament
df_grand_slam_plot['WIN LOSE PERCENTAGE'] = df_grand_slam_plot['result']/df_grand_slam_plot['Total'] * 100

# Check reuslts
df_grand_slam_plot

In [ ]:
df_grand_slam.winner_flag.value_counts()

In [ ]:
# Define colours
custom_palette = sns.color_palette(["#CD5C5C", "#FFA07A"])

# Create the bar chart
ax = sns.barplot(data=df_grand_slam_plot, x="Tournament", y="WIN LOSE PERCENTAGE", hue="winner_flag", palette=custom_palette)

# Add number labels to the bars with " %" format
for p in ax.patches:
    height = p.get_height()
    ax.annotate(f"{height:.1f} %", (p.get_x() + p.get_width() / 2, height), ha="center", va="bottom")

# Add labels and title
plt.xlabel("GRAND SLAM")
plt.ylabel("WINNING %")
plt.title("WINNING % BY GRAND SLAM")

# Rotate x-axis labels if needed
plt.xticks(rotation=45)

# Move the legend outside the plot
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Show the plot
plt.show()

In [ ]:
df_grand_slam

### Con quienes perdió mas partidos en GS

In [ ]:
df_gs_loses = df_grand_slam[df_grand_slam['winner_flag'] == 'Lose']
df_gs_loses_plot = df_gs_loses.groupby(['Winner']).agg(wins=('winner_flag','count')).reset_index().sort_values('wins', ascending=False)

In [ ]:
df_gs_loses_plot

In [ ]:
# Define colours
custom_palette = sns.color_palette(["#CD5C5C"])

# Set the figure size
plt.figure(figsize=(18, 6))  # Adjust the width (10) and height (6) as desired

# Create the bar chart
ax = sns.barplot(data=df_gs_loses_plot, x="Winner", y="wins", palette=custom_palette)

# Add number labels to the bars
for p in ax.patches:
    height = p.get_height()
    ax.annotate(f"{height:.0f}", (p.get_x() + p.get_width() / 2, height), ha="center", va="bottom")

# Add labels and title
plt.xlabel("PLAYER")
plt.ylabel("LOSSES")
plt.title("GRAND SLAM LOSSES")

# Rotate x-axis labels if needed
plt.xticks(rotation=45)

# Move the legend outside the plot
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Show the plot
plt.show()

In [ ]:
# Replicamos análisis para solamente ROLAND GARROS
df_gs_loses = df_grand_slam[(df_grand_slam['winner_flag'] == 'Lose')&(df_grand_slam['Tournament'] == 'Roland Garros')]
df_gs_loses_plot = df_gs_loses.groupby(['Winner']).agg(wins=('winner_flag','count')).reset_index().sort_values('wins', ascending=False)

# Define colours
custom_palette = sns.color_palette(["#CD5C5C"])

# Set the figure size
plt.figure(figsize=(18, 6))  # Adjust the width (10) and height (6) as desired

# Create the bar chart
ax = sns.barplot(data=df_gs_loses_plot, x="Winner", y="wins", palette=custom_palette)

# Add number labels to the bars
for p in ax.patches:
    height = p.get_height()
    ax.annotate(f"{height:.0f}", (p.get_x() + p.get_width() / 2, height), ha="center", va="bottom")

# Add labels and title
plt.xlabel("PLAYER")
plt.ylabel("LOSSES")
plt.title("ROLAND GARROS LOSSES")

# Rotate x-axis labels if needed
plt.xticks(rotation=45)

# Move the legend outside the plot
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Show the plot
plt.show()

# Mismo analisis para finales de GS

In [ ]:
df_gs_finales = df[(df['grand_slam_flag']==True)&(df['Rd']=='F')]

In [ ]:
df_gs_finales['winner_flag'].value_counts()

In [ ]:
df_grand_slam_plot = df_gs_finales.groupby(['Tournament','winner_flag']).agg(result=('Winner','count')).reset_index()

In [ ]:
# Calculate the total for each tournament
df_grand_slam_plot['Total'] = df_grand_slam_plot.groupby('Tournament')['result'].transform('sum')

# Calculate the percentage of Win and Lose for each tournament
df_grand_slam_plot['WIN LOSE PERCENTAGE'] = df_grand_slam_plot['result']/df_grand_slam_plot['Total'] * 100

# Check reuslts
df_grand_slam_plot

In [ ]:
# Define colours
custom_palette = sns.color_palette(["#CD5C5C", "#FFA07A"])

# Create the bar chart
ax = sns.barplot(data=df_grand_slam_plot, x="Tournament", y="result", hue="winner_flag", palette=custom_palette)

# Add number labels to the bars with " %" format
for p in ax.patches:
    height = p.get_height()
    ax.annotate(f"{height:.0f}", (p.get_x() + p.get_width() / 2, height), ha="center", va="bottom")

# Add labels and title
plt.xlabel("GRAND SLAM")
plt.ylabel("FINAL RESULTS")
plt.title("FINAL RESULTS BY GRAND SLAM")

# Rotate x-axis labels if needed
plt.xticks(rotation=45)

# Move the legend outside the plot
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Show the plot
plt.show()

### Con quienes perdió mas partidos en GS

In [ ]:
df_gs_loses = df_gs_finales[df_gs_finales['winner_flag'] == 'Lose']
df_gs_loses_plot = df_gs_loses.groupby(['Winner']).agg(wins=('winner_flag','count')).reset_index().sort_values('wins', ascending=False)

In [ ]:
df_gs_loses_plot

In [ ]:
# Define colours
custom_palette = sns.color_palette(["#CD5C5C"])

# Set the figure size
plt.figure(figsize=(18, 6))  # Adjust the width (10) and height (6) as desired

# Create the bar chart
ax = sns.barplot(data=df_gs_loses_plot, x="Winner", y="wins", palette=custom_palette)

# Add number labels to the bars
for p in ax.patches:
    height = p.get_height()
    ax.annotate(f"{height:.0f}", (p.get_x() + p.get_width() / 2, height), ha="center", va="bottom")

# Add labels and title
plt.xlabel("PLAYER")
plt.ylabel("LOSSES")
plt.title("FINAL LOSSES IN GS")

# Rotate x-axis labels if needed
plt.xticks(rotation=45)

# Move the legend outside the plot
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Show the plot
plt.show()

In [ ]:
df_grand_slam = df[df['grand_slam_flag']==True]

In [ ]:
df_grand_slam_plot = df_grand_slam.groupby(['Tournament','winner_flag']).agg(result=('Winner','count')).reset_index()

In [ ]:
# Calculate the total for each tournament
df_grand_slam_plot['Total'] = df_grand_slam_plot.groupby('Tournament')['result'].transform('sum')

# Calculate the percentage of Win and Lose for each tournament
df_grand_slam_plot['WIN LOSE PERCENTAGE'] = df_grand_slam_plot['result']/df_grand_slam_plot['Total'] * 100

# Check reuslts
df_grand_slam_plot

In [ ]:
# Define colours
custom_palette = sns.color_palette(["#CD5C5C", "#FFA07A"])

# Create the bar chart
ax = sns.barplot(data=df_grand_slam_plot, x="Tournament", y="WIN LOSE PERCENTAGE", hue="winner_flag", palette=custom_palette)

# Add number labels to the bars with " %" format
for p in ax.patches:
    height = p.get_height()
    ax.annotate(f"{height:.1f} %", (p.get_x() + p.get_width() / 2, height), ha="center", va="bottom")

# Add labels and title
plt.xlabel("GRAND SLAM")
plt.ylabel("WINNING %")
plt.title("WINNING % BY GRAND SLAM")

# Rotate x-axis labels if needed
plt.xticks(rotation=45)

# Move the legend outside the plot
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Show the plot
plt.show()

### Con quienes perdió mas partidos en GS

In [ ]:
df_gs_loses = df_grand_slam[df_grand_slam['winner_flag'] == 'Lose']
df_gs_loses_plot = df_gs_loses.groupby(['Winner']).agg(wins=('winner_flag','count')).reset_index().sort_values('wins', ascending=False)

In [ ]:
df_gs_loses_plot

In [ ]:
# Define colours
custom_palette = sns.color_palette(["#CD5C5C"])

# Set the figure size
plt.figure(figsize=(18, 6))  # Adjust the width (10) and height (6) as desired

# Create the bar chart
ax = sns.barplot(data=df_gs_loses_plot, x="Winner", y="wins", palette=custom_palette)

# Add number labels to the bars
for p in ax.patches:
    height = p.get_height()
    ax.annotate(f"{height:.0f}", (p.get_x() + p.get_width() / 2, height), ha="center", va="bottom")

# Add labels and title
plt.xlabel("PLAYER")
plt.ylabel("LOSSES")
plt.title("GRAND SLAM LOSSES")

# Rotate x-axis labels if needed
plt.xticks(rotation=45)

# Move the legend outside the plot
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Show the plot
plt.show()

## Historial en finales de GS

In [ ]:
df_gs_finales = df[(df['grand_slam_flag']==True)&(df['Rd']=='F')]

In [ ]:
df_gs_finales.shape

In [ ]:
df_gs_finales['rival'] = np.where(df_gs_finales['winner_flag']=='Win', df_gs_finales['Loser'], df_gs_finales['Winner'])

In [ ]:
df_historial = df_gs_finales.groupby(['rival','winner_flag']).agg(historial=('Tournament','count')).reset_index().sort_values('rival')
df_historial['total'] = df_historial.groupby(['rival'])['historial'].transform('sum')
df_historial = df_historial.sort_values(by=['winner_flag'], ascending=False).sort_values(by=['total','rival'], ascending=False)

In [ ]:
# Define colours
custom_palette = sns.color_palette(["#CD5C5C", "#FFA07A"])

# Adjust size
plt.figure(figsize=(18, 6))  # Adjust the width (10) and height (6) as desired


# Create the bar chart
ax = sns.barplot(data=df_historial, x="rival", y="historial", hue="winner_flag", palette=custom_palette)

# Add number labels to the bars with " %" format
for p in ax.patches:
    height = p.get_height()
    ax.annotate(f"{height:.0f}", (p.get_x() + p.get_width() / 2, height), ha="center", va="bottom")

# Add labels and title
plt.xlabel("RIVAL")
plt.ylabel("RECORD")
plt.title("RECORD IN GS FINALS BY PLAYER")

# Rotate x-axis labels if needed
plt.xticks(rotation=45)

# Move the legend outside the plot
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Show the plot
plt.show()

# Probabilidades por set

In [ ]:
df_grand_slam = df[df['grand_slam_flag']==True]

In [ ]:
df_grand_slam.winner_flag.value_counts()

In [ ]:
df_sets = df_grand_slam.groupby(['winner_flag']).agg(general_score=('Tournament','count')).reset_index().sort_values(by='winner_flag', ascending=False)

df_sets['total'] = df_sets['general_score'].sum()

df_sets['general_percentage'] = df_sets['general_score']/df_sets['total'] * 100

df_sets['STATE'] = 'TOTAL'

df_sets

In [ ]:
# flag de ganar el primer set
# Split the string column on the first occurrence of '-'
split_data = df_grand_slam['Score'].str.split('-', n=1, expand=True)

# Extract the characters before '-' and save them in a new column
df_grand_slam['games_1_first_set'] = split_data[0].str[-1]

# Extract the characters after '-' and save them in another new column
df_grand_slam['games_2_first_set'] = split_data[1].str[0]

In [ ]:
condlist = [(df_grand_slam['games_1_first_set'] > df_grand_slam['games_2_first_set']) & (df_grand_slam['winner_flag']=='Win'),
            (df_grand_slam['games_1_first_set'] < df_grand_slam['games_2_first_set']) & (df_grand_slam['winner_flag']=='Lose')]
choicelist = [True, True]

In [ ]:
df_grand_slam['first_set_winner'] = np.select(condlist, choicelist, False)

In [ ]:
df_sets_1 = df_grand_slam[df_grand_slam['first_set_winner']==True].groupby(['winner_flag']).agg(general_score=('Tournament','count')).reset_index().sort_values(by='winner_flag', ascending=False)

df_sets_1['total'] = df_sets_1['general_score'].sum()

df_sets_1['general_percentage'] = df_sets_1['general_score']/df_sets_1['total'] * 100

df_sets_1['STATE'] = 'FIRST SET WINNER'

df_sets_1

In [ ]:
# Analisis del ganador del segundo set
# flag de ganar el segundo set
# Split the string column on the first occurrence of '-'
split_data = df_grand_slam['Score'].str.split('-', n=2, expand=True)

# Extract the characters before '-' and save them in a new column
df_grand_slam['games_1_second_set'] = split_data[1].str[-1]

# Extract the characters after '-' and save them in another new column
df_grand_slam['games_2_second_set'] = split_data[2].str[0]

condlist = [(df_grand_slam['games_1_second_set'] > df_grand_slam['games_2_second_set']) & (df_grand_slam['winner_flag']=='Win'),
            (df_grand_slam['games_1_second_set'] < df_grand_slam['games_2_second_set']) & (df_grand_slam['winner_flag']=='Lose')]
choicelist = [True, True]

df_grand_slam['second_set_winner'] = np.select(condlist, choicelist, False)

In [ ]:
df_sets_2 = df_grand_slam[(df_grand_slam['first_set_winner']==True)&(df_grand_slam['second_set_winner']==True)].groupby(['winner_flag']).agg(general_score=('Tournament','count')).reset_index().sort_values(by='winner_flag', ascending=False)

df_sets_2['total'] = df_sets_2['general_score'].sum()

df_sets_2['general_percentage'] = df_sets_2['general_score']/df_sets_2['total'] * 100

df_sets_2['STATE'] = 'FIRST 2 SETS WINNER'

df_sets_2

In [ ]:
df_sets_total = pd.concat([df_sets, df_sets_1, df_sets_2], ignore_index=True)

# Print the concatenated DataFrame
df_sets_total

In [ ]:
# Define colours
custom_palette = sns.color_palette(["#CD5C5C", "#FFA07A"])

# Adjust size
plt.figure(figsize=(18, 6))  # Adjust the width (10) and height (6) as desired


# Create the bar chart
ax = sns.barplot(data=df_sets_total, x="STATE", y="general_percentage", hue="winner_flag", palette=custom_palette)

# Add number labels to the bars with " %" format
for p in ax.patches:
    height = p.get_height()
    ax.annotate(f"{height:.1f} %", (p.get_x() + p.get_width() / 2, height), ha="center", va="bottom")

# Add labels and title
plt.xlabel("CONDITION")
plt.ylabel("PERCENTAGE RECORD")
plt.title("RECORD PERCENTAGE IN GS")

# Rotate x-axis labels if needed
plt.xticks(rotation=45)

# Move the legend outside the plot
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Show the plot
plt.show()

In [ ]:
# Unica derrota en su carrera con 2 sets a favor
df_grand_slam[(df_grand_slam['first_set_winner']==True)&(df_grand_slam['second_set_winner']==True)&(df_grand_slam['winner_flag']=='Lose')]